# 1) Data Prep

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
cwd = os.getcwd()

counter = 0
print(counter, cwd)
while not cwd.endswith("drone_steering") and counter < 10:
    os.chdir('..')
    cwd = os.getcwd()
    counter = counter + 1
    print(counter, cwd)

0 C:\Users\p.schambach\Desktop\DSR\drone_steering\models
1 C:\Users\p.schambach\Desktop\DSR\drone_steering


## a) Data loading

In [3]:
from app_local.module import DataEnsembler, GestureTransformer

de = DataEnsembler(120)
de.investigate_available_datafiles(data_dir='data/gesture/',is_frame_based=True)
de.load_data()

## d) Generate training data

In [4]:
de.assemble_data(tolerance_range=2)

In [ ]:
de.combined_data_files_df

In [ ]:
de.display_information()

## e) Data normalization

In [5]:
X = de.X.copy()
y = de.y.copy().astype("int32")
gt = GestureTransformer(byrow=True, feature_names= list(de.feature_names))
X = gt.transform(X, verbose = True)

---------------------------------------------------------
Performing row-based transformations:
X-Mean of Hip and Shoulders:
	Row-based Mean is taken over each sample and each timestep
	Shape: (19559, 18)
Y-Mean of Hip and Shoulders:
	Row-based Mean is taken over each sample and each timestep
	Shape: (19559, 18)
Scaling based on Hip-Shoulder distance:
	Row-based distance is calculated for each sample and each timestep
	Shape: (19559, 18)
-----------------------------------------------------
X-Shift:
            min   mean  median    max   std
timestep                                   
0         196.8  295.0   282.9  511.9  37.6
1         196.8  294.9   282.8  511.9  37.6
2         196.8  294.9   282.7  511.9  37.6
3         196.8  294.9   282.7  511.9  37.6
4         196.8  294.8   282.6  511.9  37.6
5         196.8  294.8   282.6  511.9  37.6
6         196.8  294.7   282.5  511.9  37.6
7         196.8  294.7   282.5  511.9  37.6
8         196.8  294.7   282.4  511.9  37.6
9         1

# 2) Model Training

## a) Filtering 0-Labels

In [ ]:
from app_local.module import DataResampler
dr_var = DataResampler()
dr_var.fit(X,y)
dr_var.restrict_zero_labels(500, criterion="variance_lowest")
dr_var.display_information()

In [ ]:
dr = DataResampler()
dr.fit(X,y)
dr.restrict_zero_labels(500)
dr.display_information()

In [ ]:
print(dr.X.shape)
X_full_zero = np.concatenate( [dr.X, dr_var.X[(dr_var.y == 0),:,:]] )
y_full_zero = np.concatenate( [dr.y, dr_var.y[dr_var.y == 0]])
print(X_full_zero.shape, y_full_zero.shape)

In [ ]:
dr_up = DataResampler()
dr_up.fit(X_full_zero, y_full_zero)
dr_up.display_information()

In [ ]:
dr_up.upsample(n = 800, exceptions=[0,3])
dr_up.display_information()
X_final = dr_up.X
y_final = dr_up.y

## b) OneHot-Encoding of target vector

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder(sparse = False)
print(type(ohe))
target = ohe.fit_transform(y_final.reshape(-1,1))
print("Before:", y_final.shape)
print("After:", target.shape)
target[0:5,:]

## c) Train-Test-Split

## d) Model Training

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

### Train a simple SV-Classifier

In [ ]:
x_final = X_final.reshape(X_final.shape[0],-1)

model = SVC(
    random_state = 42,
    gamma = "auto"
)

model.fit(x_final, y_final)

In [ ]:
preds = model.predict(x_final)
(preds == y_final).sum()/len(y_final)

### Gridsearch with Crossvalidation

In [ ]:
model.get_params().keys()

In [ ]:
# Create the random grid vor SVM

random_grid = {
                'C': [100,10,5.0,2.0,1.0, 0.5, 0.1, 0.5, 0.01],
                'gamma': ["auto", 0.001, 0.0001],
                'kernel': ["rbf","poly"]
              }

print(random_grid)

In [ ]:
params = random_grid
scoring = "accuracy"
clf = GridSearchCV(model, params, n_jobs = -1, verbose = 2, scoring = scoring)
clf.fit(x_final, y_final)

In [ ]:
clf.cv_results_.keys()

In [ ]:
print("Index: {0}".format(clf.best_index_))
print("Score: {0}".format(clf.best_score_))
print("Params: {0}".format(clf.best_params_))

In [ ]:
from sklearn.externals import joblib
joblib.dump(clf.best_estimator_,'models/model_gesture_svm_nip120_[1000,800]_002.pkl')

In [6]:
from sklearn.externals import joblib
import os
cwd = os.getcwd()

counter = 0
print(counter, cwd)
while not cwd.endswith("drone_steering") and counter < 10:
    os.chdir('..')
    cwd = os.getcwd()
    counter = counter + 1
    print(counter, cwd)

loaded_model = joblib.load('models/model_gesture_svm_nip120_[1000,800]_001.pkl')
print(type(loaded_model))

0 C:\Users\p.schambach\Desktop\DSR\drone_steering
<class 'sklearn.model_selection._search.GridSearchCV'>


In [12]:
print(loaded_model.predict(X[6,:,:].reshape(1,-1)))
x6 = X[6,:,:].reshape(1,-1)

[3]


In [13]:
joblib.dump(x6,"models/x6.pkl")

['models/x6.pkl']

In [14]:
from sklearn.externals import joblib
joblib.dump(loaded_model.best_estimator_,'models/model_gesture_svm_nip120_[1000,800]_001.pkl')

['models/model_gesture_svm_nip120_[1000,800]_001.pkl']